In [1]:
import os
os.chdir('/Users/grace/cs6742-fork/convokit/SRILM')
os.listdir()

['srilm-1.7.3',
 'dump',
 '.DS_Store',
 'srilm-demo.ipynb',
 'lm_training',
 'LMTransformer.py',
 '__init__.py',
 '__pycache__',
 '.ipynb_checkpoints',
 'srilmWrapper.py']

In [2]:
import sys
sys.path.append(os.getcwd())

In [3]:
from srilmWrapper import LanguageModel

In [4]:
os.getcwd()

'/Users/grace/cs6742-fork/convokit/SRILM'

In [5]:
from tqdm import tqdm
import os
os.chdir('/Users/grace/cs6742-fork/')
from convokit import Corpus, User, Utterance #Removed empath from init.py, file not found
from convokit import PolTracker
import nltk
import numpy as np
from collections import defaultdict
import seaborn as sns
import pandas as pd
from itertools import product

In [6]:
from pandas import Series
from matplotlib import pyplot
from statsmodels.tsa.seasonal import seasonal_decompose
import json
from nltk import word_tokenize

## Importing Pro vs. Anti Gun LM - Trained on Gun Forum Data

In [7]:
lm_antigun = LanguageModel(SRILM_path='/Users/grace/cs6742-fork/convokit/SRILM/srilm-1.7.3',
                  working_dir='/Users/grace/cs6742-fork/convokit/SRILM/dump/',
                  lm_output_path='antigun_forum_full.lm',
                  lm_type='laplace',
                  count_output_path='antigun_forum_full_counts.txt',
                  order=2,
                  lm_filepath='/Users/grace/cs6742-fork/convokit/SRILM/dump/antigun_forum_full.lm',
                  verbose=False)

In [8]:
lm_progun  = LanguageModel(SRILM_path='/Users/grace/cs6742-fork/convokit/SRILM/srilm-1.7.3',
                  working_dir='/Users/grace/cs6742-fork/convokit/SRILM/dump/',
                  lm_output_path='progun_forum_full.lm',
                  lm_type='laplace',
                  count_output_path='progun_forum_full_counts.txt',
                  order=2,
                  lm_filepath='/Users/grace/cs6742-fork/convokit/SRILM/dump/progun_forum_full.lm',
                  verbose=False)

In [9]:
lm_gun = LanguageModel(SRILM_path='/Users/grace/cs6742-fork/convokit/SRILM/srilm-1.7.3',
                  working_dir='/Users/grace/cs6742-fork/convokit/SRILM/dump/',
                  lm_output_path='whole_forum_full.lm',
                  lm_type='laplace',
                  count_output_path='whole_forum_full_counts.txt',
                  order=2,
                  lm_filepath='/Users/grace/cs6742-fork/convokit/SRILM/dump/whole_forum_full.lm',
                  verbose=False)

#### Sanity check for scores for typical anti vs. pro gun phrases

In [10]:
lm_antigun.str_perplexity("This is a mental health issue.")

797.9711

In [11]:
lm_progun.str_perplexity("This is a mental health issue.")

838.7635

In [12]:
lm_antigun.str_perplexity1("This is a mental health issue.")

2430.254

In [13]:
lm_progun.str_perplexity1("This is a mental health issue .")

2303.476

## Applying To Subreddit Groups

Reading in CSV for 3 groups of subreddits with time_bins added, and repeating users removed 
This now uses the subreddits_round2 after data has been cleaned using clean-text module

In [14]:
import pandas as pd

#### Liberal Subreddits

In [15]:
os.chdir('/Users/grace/Desktop/CS6742/subreddits_round2/')

In [16]:
%%time

liberal_group = pd.read_csv('liberal_cleaned.csv')

CPU times: user 40.2 ms, sys: 11.5 ms, total: 51.7 ms
Wall time: 58.1 ms


In [17]:
liberal_group.shape

(4231, 10)

In [18]:
%%time
liberal_group['antigun_entropy'] = [lm_antigun.str_perplexity1(i) for i in liberal_group['text_30tokens']]

CPU times: user 2.44 s, sys: 12.3 s, total: 14.8 s
Wall time: 3min 22s


In [19]:
%%time
liberal_group['progun_entropy'] = [lm_progun.str_perplexity1(i) for i in liberal_group['text_30tokens']] 

CPU times: user 2.45 s, sys: 12.6 s, total: 15 s
Wall time: 3min 44s


In [20]:
%%time
liberal_group['neutral_entropy'] = [lm_gun.str_perplexity1(i) for i in liberal_group['text_30tokens']] 

CPU times: user 2.38 s, sys: 12.1 s, total: 14.5 s
Wall time: 8min 13s


In [21]:
liberal_group.head(5)

,X,utterid,username,subreddit,event,timestamp,text,text_30tokens,time_bin,user_timebin,antigun_entropy,progun_entropy,neutral_entropy
0,4,14vbrx,[deleted],Liberal,Sandy Hook Elementary School shooting,1355529589,ill preface this by saying im canadian i live ...,ill preface this by saying im canadian i live ...,2.0,[deleted] 2 Sandy Hook Elementary School shooting,2807.050,4531.560,3430.337
1,5,14w7rp,[deleted],Liberal,Sandy Hook Elementary School shooting,1355581147,curious as to your thoughts reddit i do think ...,curious as to your thoughts reddit i do think ...,3.0,[deleted] 3 Sandy Hook Elementary School shooting,3025.056,3986.197,3894.668
2,6,14vlel,[deleted],Liberal,Sandy Hook Elementary School shooting,1355540354,general sentiment seems to be gun control isnt...,general sentiment seems to be gun control isnt...,2.0,[deleted] 2 Sandy Hook Elementary School shooting,1905.938,2795.577,3606.641
3,7,15humz,[deleted],Liberal,Sandy Hook Elementary School shooting,1356572203,i want to understand why and how gun control w...,i want to understand why and how gun control w...,26.0,[deleted] 26 Sandy Hook Elementary School shoo...,4620.227,3264.820,5145.403
4,10,c5hicy4,[deleted],Liberal,Aurora theater shooting,1342927769,i disagree with the article choosing to politi...,i disagree with the article choosing to politi...,4.0,[deleted] 4 Aurora theater shooting,1763.416,2727.233,4318.033


In [22]:
%%time
liberal_group.to_csv("/Users/grace/Desktop/CS6742/subreddits_round2/liberal_entropy.csv")

CPU times: user 95 ms, sys: 12.1 ms, total: 107 ms
Wall time: 106 ms


In [23]:
#del liberal_group

#### Now Conservative Group LM Model

In [24]:
os.chdir('/Users/grace/Desktop/CS6742/subreddits_round2/')

In [25]:
%%time
#conservative_group = pd.read_csv('conservative_uniquser_newsample.csv')
conservative_group = pd.read_csv('conservative_cleaned.csv')
print(conservative_group.shape)

(17747, 10)
CPU times: user 125 ms, sys: 24.2 ms, total: 150 ms
Wall time: 149 ms


In [26]:
%%time
antigun_entropy = []

for i in list(range(0, len(conservative_group['text_30tokens']))):
    entropy = lm_antigun.str_perplexity1(conservative_group['text_30tokens'][i])
    if i % 500 == 0:
        print([i, entropy])
    antigun_entropy.append(entropy)
    

[0, 2296.215]
[500, 1477.612]
[1000, 2632.563]
[1500, 1581.524]
[2000, 2524.935]
[2500, 2012.993]
[3000, 2479.944]
[3500, 3112.869]
[4000, 2634.698]
[4500, 3902.967]
[5000, 2606.819]
[5500, 3983.672]
[6000, 3852.062]
[6500, 2389.38]
[7000, 5070.394]
[7500, 3670.242]
[8000, 2898.382]
[8500, 2494.611]
[9000, 2002.966]
[9500, 2147.46]
[10000, 3518.784]
[10500, 1696.5]
[11000, 3089.213]
[11500, 3201.424]
[12000, 2372.348]
[12500, 2888.199]
[13000, 2899.974]
[13500, 1574.537]
[14000, 3952.365]
[14500, 2938.283]
[15000, 3055.61]
[15500, 1964.881]
[16000, 2464.205]
[16500, 958.5705]
[17000, 1543.947]
[17500, 1719.936]
CPU times: user 13.7 s, sys: 1min 1s, total: 1min 15s
Wall time: 16min 20s


In [27]:
conservative_group['antigun_entropy'] = antigun_entropy

In [28]:
%%time
progun_entropy = []
for i in list(range(0, len(conservative_group['text_30tokens']))):
    entropy = lm_progun.str_perplexity1(conservative_group['text_30tokens'][i])
    if i % 500 == 0:
        print([i, entropy])
    progun_entropy.append(entropy)
    

[0, 5078.898]
[500, 3909.833]
[1000, 2473.592]
[1500, 2680.335]
[2000, 5782.463]
[2500, 5907.584]
[3000, 4113.695]
[3500, 3600.654]
[4000, 2521.314]
[4500, 5604.356]
[5000, 4231.053]
[5500, 4966.276]
[6000, 5070.562]
[6500, 3993.496]
[7000, 9855.075]
[7500, 2915.314]
[8000, 3809.854]
[8500, 2706.516]
[9000, 3532.829]
[9500, 6653.156]
[10000, 7930.731]
[10500, 5411.628]
[11000, 3703.721]
[11500, 3969.071]
[12000, 2891.1]
[12500, 3739.035]
[13000, 4828.674]
[13500, 2462.37]
[14000, 6079.356]
[14500, 6897.311]
[15000, 8930.549]
[15500, 7644.141]
[16000, 4483.963]
[16500, 3404.977]
[17000, 4690.708]
[17500, 2615.273]
CPU times: user 12.8 s, sys: 56.1 s, total: 1min 8s
Wall time: 16min 21s


In [30]:
%%time
conservative_group['progun_entropy'] = progun_entropy

CPU times: user 1.61 ms, sys: 244 µs, total: 1.85 ms
Wall time: 1.79 ms


In [31]:
%%time
neutral_entropy = []
for i in list(range(0, len(conservative_group['text_30tokens']))):
    entropy = lm_gun.str_perplexity1(conservative_group['text_30tokens'][i])
    if i % 500 == 0:
        print([i, entropy])
    neutral_entropy.append(entropy)

[0, 6681.379]
[500, 3715.132]
[1000, 2371.695]
[1500, 2219.007]
[2000, 5956.105]
[2500, 4632.029]
[3000, 3265.51]
[3500, 5568.731]
[4000, 3449.814]
[4500, 8158.869]
[5000, 4738.624]
[5500, 5891.28]
[6000, 6103.651]
[6500, 6651.209]
[7000, 10505.1]
[7500, 2238.571]
[8000, 3957.524]
[8500, 3165.594]
[9000, 4547.149]
[9500, 8559.172]
[10000, 9850.129]
[10500, 5196.503]
[11000, 3567.399]
[11500, 5520.247]
[12000, 4202.088]
[12500, 3329.948]
[13000, 4794.546]
[13500, 3496.27]
[14000, 6762.265]
[14500, 7347.641]
[15000, 6358.441]
[15500, 7808.626]
[16000, 4559.046]
[16500, 3123.448]
[17000, 4522.254]
[17500, 3749.262]
CPU times: user 12.9 s, sys: 53.8 s, total: 1min 6s
Wall time: 37min 1s


In [33]:
%%time
conservative_group['neutral_entropy'] = neutral_entropy

CPU times: user 1.55 ms, sys: 224 µs, total: 1.78 ms
Wall time: 1.86 ms


In [35]:
%%time
conservative_group.to_csv("/Users/grace/Desktop/CS6742/subreddits_round2/conservative_entropy.csv")

CPU times: user 349 ms, sys: 44.2 ms, total: 393 ms
Wall time: 398 ms


In [34]:
conservative_group.head(5)

,X,utterid,username,subreddit,event,timestamp,text,text_30tokens,time_bin,user_timebin,antigun_entropy,progun_entropy,neutral_entropy
0,6260,d46izuy,tsdouglas,The_Donald,Orlando nightclub shooting,1465769039,remember this same president is responsible fo...,remember this same president is responsible fo...,2.0,tsdouglas 2,2296.215,5078.898,6681.379
1,5988,d46h223,theorymeltfool,The_Donald,Orlando nightclub shooting,1465766020,id rather just buy another gun opencarrying as...,id rather just buy another gun opencarrying as...,2.0,theorymeltfool 2,2929.253,2093.525,4789.977
2,4138,d466i9t,SkaggAteMyPappy,The_Donald,Orlando nightclub shooting,1465751932,anyone whos lgbtq and supports islam is litera...,anyone whos lgbtq and supports islam is litera...,2.0,SkaggAteMyPappy 2,2458.871,4161.457,5268.886
3,5427,d46dsxa,arsenal12ful,The_Donald,Orlando nightclub shooting,1465762138,now i see what trump meant when he said not al...,now i see what trump meant when he said not al...,2.0,arsenal12ful 2,1673.210,2643.810,4180.132
4,3154,d468a71,Cannon1,Conservative,Orlando nightclub shooting,1465754403,it really is interesting and quite scary that ...,it really is interesting and quite scary that ...,2.0,Cannon1 2,1948.113,2849.631,2383.255


From 2012 ... That aside , far be it from me to pretend mature Japanese women are n't appealing but that 's pretty ridiculous . Degeneracy is what liberals are
From 2012...That aside, far be it from me to pretend mature Japanese women aren't appealing but that's pretty ridiculous . Degeneracy is what liberals are
From 2012... That aside, far be it from me to pretend mature Japanese women aren't appealing but that's pretty ridiculous. Degeneracy is what liberals are all about no matter the


2

X                                                            11079
utterid                                                    duhsepv
username                                            DogBeersHadOne
subreddit                                             Conservative
event                        Stoneman Douglas High School shooting
timestamp                                               1519054870
text             \#1 is already on the books.\n\n\#2 would have...
text_30tokens                                                     
time_bin                                                        44
user_timebin                                     DogBeersHadOne 44
Name: 9341, dtype: object

#### Neutral Group Subreddits (Need to Stratify Sample to reduce to 10% of sample size)

In [36]:
%%time
neutral_group = pd.read_csv('neutral_cleaned.csv')
print(neutral_group.shape)

(28009, 10)
CPU times: user 219 ms, sys: 34.1 ms, total: 253 ms
Wall time: 252 ms


In [37]:
neutral_group.head(4)

,X,utterid,username,subreddit,event,timestamp,text,text_30tokens,time_bin,user_timebin
0,14411,c7grtof,[deleted],worldnews,Sandy Hook Elementary School shooting,1355532166,i can definitely agree that it would be very h...,i can definitely agree that it would be very h...,2.0,[deleted] 2
1,33561,c7gotoq,[deleted],politics,Sandy Hook Elementary School shooting,1355521177,> guns are not magic wands i completely agree ...,> guns are not magic wands i completely agree ...,2.0,[deleted] 2
2,31710,c7gnkpo,[deleted],politics,Sandy Hook Elementary School shooting,1355517205,who who in their right mind could say that chi...,who who in their right mind could say that chi...,2.0,[deleted] 2
3,8649,c7gkwjf,[deleted],news,Sandy Hook Elementary School shooting,1355508739,can i clear up if you are talking about size o...,can i clear up if you are talking about size o...,2.0,[deleted] 2


In [38]:
%%time
antigun_entropy = []

for i in list(range(0, len(neutral_group['text_30tokens']))):
    entropy = lm_antigun.str_perplexity1(neutral_group['text_30tokens'][i])
    if i % 500 == 0:
        print([i, entropy])
    antigun_entropy.append(entropy)
    

[0, 1989.305]
[500, 1895.539]
[1000, 1954.866]
[1500, 933.0778]
[2000, 3244.047]
[2500, 2200.27]
[3000, 1704.224]
[3500, 1348.817]
[4000, 2632.048]
[4500, 2566.3]
[5000, 2291.137]
[5500, 3396.38]
[6000, 2974.207]
[6500, 2104.4]
[7000, 1727.663]
[7500, 2834.462]
[8000, 1845.715]
[8500, 2192.799]
[9000, 2070.729]
[9500, 2883.788]
[10000, 2276.122]
[10500, 2889.495]
[11000, 1753.418]
[11500, 3638.822]
[12000, 3290.787]
[12500, 1895.86]
[13000, 1776.086]
[13500, 2324.515]
[14000, 1668.291]
[14500, 4145.433]
[15000, 2491.321]
[15500, 2461.856]
[16000, 5606.38]
[16500, 1753.924]
[17000, 2626.205]
[17500, 1841.211]
[18000, 2540.53]
[18500, 2132.822]
[19000, 2286.738]
[19500, 1903.971]
[20000, 1950.216]
[20500, 3019.31]
[21000, 2943.217]
[21500, 3624.485]
[22000, 1868.62]
[22500, 2040.653]
[23000, 2263.327]
[23500, 1553.335]
[24000, 1315.712]
[24500, 2114.643]
[25000, 1746.921]
[25500, 2838.116]
[26000, 1583.881]
[26500, 2098.544]
[27000, 1727.808]
[27500, 2219.176]
[28000, 2029.677]
CPU times

In [39]:
neutral_group['antigun_entropy'] = antigun_entropy

In [40]:
%%time
progun_entropy = []

for i in list(range(0, len(neutral_group['text_30tokens']))):
    entropy = lm_progun.str_perplexity1(neutral_group['text_30tokens'][i])
    if i % 500 == 0:
        print([i, entropy])
    progun_entropy.append(entropy)

[0, 1964.99]
[500, 8384.985]
[1000, 2298.518]
[1500, 1149.881]
[2000, 5493.596]
[2500, 3322.541]
[3000, 2133.53]
[3500, 2770.316]
[4000, 6161.766]
[4500, 3644.887]
[5000, 3170.911]
[5500, 4564.447]
[6000, 2861.471]
[6500, 5319.192]
[7000, 2673.842]
[7500, 4442.288]
[8000, 2902.92]
[8500, 5125.647]
[9000, 3204.808]
[9500, 5029.423]
[10000, 4021.459]
[10500, 4161.523]
[11000, 2937.257]
[11500, 5108.519]
[12000, 3280.02]
[12500, 3957.596]
[13000, 2507.118]
[13500, 2946.439]
[14000, 3642.038]
[14500, 5390.107]
[15000, 5002.766]
[15500, 3575.472]
[16000, 12919.71]
[16500, 5190.408]
[17000, 3328.255]
[17500, 4819.131]
[18000, 9017.351]
[18500, 2709.402]
[19000, 3635.389]
[19500, 1921.385]
[20000, 2693.451]
[20500, 8834.072]
[21000, 3841.849]
[21500, 3993.928]
[22000, 2218.183]
[22500, 1982.617]
[23000, 2429.845]
[23500, 2727.508]
[24000, 4976.304]
[24500, 4330.853]
[25000, 2480.123]
[25500, 2401.841]
[26000, 2092.614]
[26500, 6721.126]
[27000, 3417.518]
[27500, 3529.581]
[28000, 3263.824]
CP

In [41]:
neutral_group['progun_entropy'] = progun_entropy

In [42]:
%%time
neutral_entropy = []

for i in list(range(0, len(neutral_group['text_30tokens']))):
    entropy = lm_gun.str_perplexity1(neutral_group['text_30tokens'][i])
    if i % 500 == 0:
        print([i, entropy])
    neutral_entropy.append(entropy)

[0, 2522.948]
[500, 10709.82]
[1000, 2711.366]
[1500, 1663.91]
[2000, 4088.675]
[2500, 4351.929]
[3000, 2120.194]
[3500, 3683.095]
[4000, 6688.683]
[4500, 3152.954]
[5000, 3493.199]
[5500, 6010.023]
[6000, 4110.243]
[6500, 5734.316]
[7000, 3844.553]
[7500, 4025.547]
[8000, 3915.315]
[8500, 6432.494]
[9000, 3298.631]
[9500, 5261.991]
[10000, 4694.388]
[10500, 4581.65]
[11000, 3761.293]
[11500, 6843.87]
[12000, 6392.756]
[12500, 3715.244]
[13000, 2826.218]
[13500, 3714.656]
[14000, 5439.271]
[14500, 6678.314]
[15000, 7234.021]
[15500, 3320.252]
[16000, 15260.85]
[16500, 6440.162]
[17000, 3573.571]
[17500, 5196.884]
[18000, 8566.954]
[18500, 2737.059]
[19000, 3459.373]
[19500, 3014.256]
[20000, 2895.936]
[20500, 8361.228]
[21000, 4534.035]
[21500, 4720.686]
[22000, 2905.476]
[22500, 2754.607]
[23000, 2670.687]
[23500, 2922.933]
[24000, 3402.796]
[24500, 4426.114]
[25000, 2306.517]
[25500, 2682.404]
[26000, 3278.461]
[26500, 7954.973]
[27000, 3476.215]
[27500, 4499.437]
[28000, 2757.277]
C

In [43]:
neutral_group['neutral_entropy'] = neutral_entropy

In [46]:
neutral_group.to_csv("/Users/grace/Desktop/CS6742/subreddits_round2/neutral_entropy.csv")